<a href="https://colab.research.google.com/github/raymondwcs/learning_bert/blob/master/Cluster_Sentence_Embedding_(kMeans%2C_DBSCAN%2C_sentence_transformers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers
!pip install --quiet sentence-transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
     |████████████████████████████████| 636 kB 40.1 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 11.7 MB/s 


In [2]:
# from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# CHECKPOINT = 'bert-base-chinese'
CHECKPOINT = 'distiluse-base-multilingual-cased-v2'

MAX_LENGTH = 80

# corpus = [
#   "Vodafone Wins ₹20,000 Crore Tax Arbitration Case Against Government",
#   "Voda Idea shares jump nearly 15% as Vodafone wins retro tax case in Hague",
#   "Gold prices today fall for 4th time in 5 days, down ₹6500 from last month high",
#   "Silver futures slip 0.36% to Rs 59,415 per kg, down over 12% this week",
#   "Amazon unveils drone that films inside your home. What could go wrong?",
#   "IPHONE 12 MINI PERFORMANCE MAY DISAPPOINT DUE TO THE APPLE B14 CHIP",
#   "Delhi Capitals vs Chennai Super Kings: Prithvi Shaw shines as DC beat CSK to post second consecutive win in IPL",
#   "French Open 2020: Rafael Nadal handed tough draw in bid for record-equaling 20th Grand Slam"
# ]

corpus = [
  '這個服務生很不親切',         
  '这个服务生很不亲切', 
  '黄昏時滂沱大雨',  
  '放工時下大雨',
  '這個週末陽光普照!',
  '天朗氣清的星期天。',
  '現時恒指已跌穿重要支持位26500點來看。',
  '港股繼續尋底的機會是頗高的。'         
]
corpus_embedding = []

# tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
# model = AutoModel.from_pretrained(CHECKPOINT)
# tokens = tokenizer(text=corpus, max_length=MAX_LENGTH, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt')
# output = model(**tokens)
# for i in range(len(output.pooler_output)):
#   corpus_embedding.append(output.pooler_output[i].detach().numpy())

embedder = SentenceTransformer(CHECKPOINT)
corpus_embedding = embedder.encode(corpus)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/610 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

# Mesaure Embeddings Similarity

In [3]:
similarities = cosine_similarity(corpus_embedding)
similarities_sorted = similarities.argsort()
id_1 = []
id_2 = []
score = []
for index,array in enumerate(similarities_sorted):
    id_1.append(index)
    id_2.append(array[-2])
    score.append(similarities[index][array[-2]])
index_df = pd.DataFrame({'id_1' : id_1,
                          'id_2' : id_2,
                          'score' : score})

print(index_df)

   id_1  id_2     score
0     0     1  0.953719
1     1     0  0.953719
2     2     3  0.841035
3     3     2  0.841035
4     4     5  0.746492
5     5     4  0.746492
6     6     7  0.181023
7     7     6  0.181023


# K-Means Clustering (distance)

In [4]:
from sklearn.cluster import KMeans

num_clusters = 4
# Define kmeans model
clustering_model = KMeans(n_clusters=num_clusters)
# Fit the embedding with kmeans clustering.
clustering_model.fit(corpus_embedding)
# Get the cluster id assigned to each news headline.
cluster_assignment = clustering_model.labels_

print(cluster_assignment)

[2 2 1 1 1 1 3 0]


In [5]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['港股繼續尋底的機會是頗高的。']

Cluster  2
['黄昏時滂沱大雨', '放工時下大雨', '這個週末陽光普照!', '天朗氣清的星期天。']

Cluster  3
['這個服務生很不親切', '这个服务生很不亲切']

Cluster  4
['現時恒指已跌穿重要支持位26500點來看。']



# K-Means Clustering (similarity)

In [6]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer
num_clusters = 4
data = corpus_embedding
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
cluster_assignment = kclusterer.cluster(data, assign_clusters=True)

print(cluster_assignment)
print()

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

[1, 1, 0, 0, 2, 2, 3, 3]

Cluster  1
['黄昏時滂沱大雨', '放工時下大雨']

Cluster  2
['這個服務生很不親切', '这个服务生很不亲切']

Cluster  3
['這個週末陽光普照!', '天朗氣清的星期天。']

Cluster  4
['現時恒指已跌穿重要支持位26500點來看。', '港股繼續尋底的機會是頗高的。']



# DBSCAN (distance)

In [15]:
# from sklearn.cluster import DBSCAN

# cluster = DBSCAN(eps=1,min_samples=1).fit(corpus_embedding)
# cluster.labels_

array([0, 0, 0, 0, 0, 0, 0, 0])

In [22]:
!pip install --quiet hdbscan

import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=2,metric='euclidean',cluster_selection_method='eom').fit(corpus_embedding)
cluster.labels_


array([ 0,  0,  1,  1,  2,  2, -1, -1])